Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [2]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
from tqdm import tqdm
import matplotlib.pyplot as plt

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [3]:
def setup_data(filen):
    cities = pd.read_csv(filen, header=None, names=['name', 'lat', 'lon'])
    num_cities = len(cities)
    dist_matrix = np.zeros((len(cities), len(cities)))
    for c1, c2 in combinations(cities.itertuples(), 2):
        dist_matrix[c1.Index, c2.Index] = dist_matrix[c2.Index, c1.Index] = geodesic(
            (c1.lat, c1.lon), (c2.lat, c2.lon)
        ).km
    return cities, dist_matrix

## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [4]:
def tsp_cost(tsp):
    sol = tsp.copy()
    if sol[0] != sol[-1]:
        sol = sol + [sol[0]]
    assert sol[0] == sol[-1]
    #assert set(sol) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(sol, sol[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost


## Greedy Algorithm

In [5]:
def greedy_tsp(cities, dist_matrix):
    visited = np.full(len(cities), False)
    dist = dist_matrix.copy()
    city = 0
    visited[city] = True
    tsp = list()
    tsp.append(int(city))
    while not np.all(visited):
        dist[:, city] = np.inf
        closest = np.argmin(dist[city])
        # logging.debug(
        #     f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city,closest]:.2f}km)"
        # )
        visited[closest] = True
        city = closest
        tsp.append(int(city))
    # logging.debug(
    #     f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATRIX[tsp[-1],tsp[0]]:.2f}km)"
    # )
    
    #tsp.append(tsp[0])
    

    #logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")
    
    return tsp, tsp_cost(tsp)

## Evolutionary



In [6]:
def invertion(solution):
    i1 = np.random.randint(0, len(solution))
    i2 = np.random.randint(0, len(solution))
    if i1 > i2:
        i1, i2 = i2, i1
    cut = solution[i1:i2+1]
    sol_copy = solution.copy()
    if i1==0:
        sol_copy[i2::-1] = cut
    else:
        sol_copy[i2 :i1 -1: -1] = cut
    return sol_copy 

def tweak(solution):
    pm = 1/ len(solution)
    sol_copy = solution.copy()
    p = None
    while p is None or p < pm:
        i1 = np.random.randint(0, len(solution))
        i2 = np.random.randint(0, len(solution))
        sol_copy[i1], sol_copy[i2] = sol_copy[i2], sol_copy[i1]
        p = np.random.random()
        
    return sol_copy   

def scramble_mutation(solution):
    SIGMA = 0.5
    sol_copy = np.array(solution.copy())
    mask = np.random.random(len(sol_copy)) < SIGMA
    sol_copy[mask] = np.random.permutation(sol_copy[mask])
    return sol_copy.tolist()

def parent_selection(population):
    sorted_population_best = sorted(population, key=lambda x: tsp_cost(x))[:10]
    return sorted_population_best[np.random.randint(0,10)]
    

def inver_over(p1,p2):
    # Select first gene
    i1= np.random.randint(0, len(p1))
    g1 = p1[i1]
    i_of_g1_in_p2= p2.index(g1)
    if i_of_g1_in_p2 +1 >= len(p2):
        g2 = p2[0]
    else:
        g2 = p2[i_of_g1_in_p2 +1]
    
    i2 = p1.index(g2)
    
    # Now i1 and i2 are the index of g1 and g2 in the first parent
    xo = None
    
    p1 = p1[i1:] + p1[:i1]
        
    i1 = p1.index(g1)
    i2 = p1.index(g2)
    cut = p1[i1+1: i2]
    cut.reverse()
    if len(cut) > 0:
        xo = [g1] + [g2] + cut + p1[i2+1:]
        return xo
    return p1

In [ ]:
def ea_tsp(cities, generations, seed = None):
    POPULATION_SIZE = len(cities) // 2 + 7
    population = [[i for i in range(len(cities))] for _ in range(POPULATION_SIZE)]
    for p in population:
        np.random.shuffle(p)
    if seed != None:
    
        population.append(seed)


    champion = min(population, key=lambda x: tsp_cost(x))
    champion_cost = tsp_cost(champion)
    
    OFFSPRING_SIZE = int(round(0.66 * POPULATION_SIZE))

    for _ in tqdm(range(generations)):
        offspring = []
        for _ in range(OFFSPRING_SIZE):
            p1 = parent_selection(population)
            p2 = parent_selection(population)
            
            offspring.append(tweak(p1))
            if np.random.rand() < 0.9:
                # Mutation
                o = invertion(p1)
                offspring.append(o)
            else:
                offspring.append(inver_over(p1,p2))
        
        population.extend(offspring)
    
        #population = sorted(population, key=lambda x: tsp_cost(x))[:POPULATION_SIZE]
        population.sort( key= lambda x : tsp_cost(x))
        population = population[:POPULATION_SIZE]
        offspring_champion = min(population, key=lambda x: tsp_cost(x))
        offspring_cost = tsp_cost(offspring_champion)
        if offspring_cost < champion_cost:

            champion = offspring_champion
            champion_cost = offspring_cost

            
    return champion, champion_cost
          


## Local search


In [8]:
def simulated_annealing(starting_point, temp, alpha, max_iter):
    current_solution = starting_point
    current_cost = tsp_cost(current_solution)
    best_solution = current_solution
    best_cost = current_cost
    for i in tqdm(range(max_iter)):
        while np.random.rand() < 0.3:
            new_solution = tweak(current_solution)
        new_solution = invertion(current_solution)
        new_cost = tsp_cost(new_solution)
        if new_cost < current_cost:
            current_solution = new_solution
            current_cost = new_cost
            if new_cost < best_cost:
                best_solution = new_solution
                best_cost = new_cost
        else:
            if np.random.rand() < np.exp((current_cost - new_cost) / temp):
                current_solution = new_solution
                current_cost = new_cost
        temp *= alpha
    return best_solution, best_cost

## Results

In [40]:
# Tested with:
# POPULATION_SIZE = 100
# GENERATIONS = 5000

# filenames = ["cities/vanuatu.csv", "cities/italy.csv", "cities/russia.csv", "cities/us.csv"]
# global CITIES, DIST_MATRIX
# for filen in filenames:
#     instance = filen.split("/")[1].replace(".csv","")
#     ic(filen)
#     cities, dist_matrix = setup_data(filen)
#     CITIES = cities
#     DIST_MATRIX = dist_matrix
#     GREEDY_TSP, GREEDY_COST = greedy_tsp(cities, dist_matrix)
#     np.save(instance + "_greedy_tsp", GREEDY_TSP)
#     SIM_AN_SOL, SIM_AN_COST = simulated_annealing( GREEDY_TSP, 100, 0.999, 100000)
#     np.save(instance + "_localsearch_tsp", SIM_AN_SOL)
#     EA_TSP, EA_COST = ea_tsp(cities)
#     np.save(instance + "_ea_tsp", EA_TSP)
#     ic(GREEDY_COST,SIM_AN_COST, EA_COST)
    
# None


In [43]:
filen= 'cities/vanuatu.csv'
instance = filen.split("/")[1].replace(".csv","")
ic(filen)
cities, dist_matrix = setup_data(filen)
CITIES = cities
DIST_MATRIX = dist_matrix

GREEDY_TSP, GREEDY_COST = greedy_tsp(cities, dist_matrix)
#np.save(instance + "_greedy_tsp", GREEDY_TSP)

SIM_AN_SOL, SIM_AN_COST = simulated_annealing( GREEDY_TSP, 100, 0.999, 100000)
#np.save(instance + "_localsearch_tsp", SIM_AN_SOL)

GENERATIONS = 1000 # Few cities, faster to optimize to global optimum
EA_TSP, EA_COST = ea_tsp(cities, GENERATIONS)
#np.save(instance + "_ea_tsp", EA_TSP)

ic(GREEDY_COST,SIM_AN_COST, EA_COST)
None

ic| filen: 'cities/vanuatu.csv'
100%|██████████| 1000/1000 [00:00<00:00, 2640.23it/s]
ic| GREEDY_COST: np.float64(1475.528091104531)
    SIM_AN_COST: np.float64(1345.544956473311)
    EA_COST: np.float64(1345.544956473311)


In [ ]:
filen= 'cities/italy.csv'
instance = filen.split("/")[1].replace(".csv","")
ic(filen)
cities, dist_matrix = setup_data(filen)
CITIES = cities
DIST_MATRIX = dist_matrix

GREEDY_TSP, GREEDY_COST = greedy_tsp(cities, dist_matrix)
np.save(instance + "_greedy_tsp", GREEDY_TSP)

SIM_AN_SOL, SIM_AN_COST = simulated_annealing( GREEDY_TSP, 100, 0.999, 100000)
np.save(instance + "_localsearch_tsp", SIM_AN_SOL)

GENERATIONS= 3000
EA_TSP, EA_COST = ea_tsp(cities, GENERATIONS)
np.save(instance + "_ea_tsp", EA_TSP)
ic(GREEDY_COST,SIM_AN_COST, EA_COST)
None

ic| filen: 'cities/italy.csv'
100%|██████████| 3000/3000 [00:28<00:00, 104.71it/s]
ic| GREEDY_COST: np.float64(4436.031769525158)
    SIM_AN_COST: np.float64(4182.7217050918125)
    EA_COST: np.float64(4182.7217050918125)


In [62]:
filen= 'cities/russia.csv'
instance = filen.split("/")[1].replace(".csv","")
ic(filen)
cities, dist_matrix = setup_data(filen)
CITIES = cities
DIST_MATRIX = dist_matrix
GREEDY_TSP, GREEDY_COST = greedy_tsp(cities, dist_matrix)
np.save(instance + "_greedy_tsp", GREEDY_TSP)
SIM_AN_SOL, SIM_AN_COST = simulated_annealing( GREEDY_TSP, 100, 0.999, 100000)
np.save(instance + "_localsearch_tsp", SIM_AN_SOL)
GENERATIONS = 2000
EA_TSP, EA_COST = ea_tsp(cities, GENERATIONS)
np.save(instance + "_ea_tsp", EA_TSP)
ic(GREEDY_COST,SIM_AN_COST, EA_COST)
None

ic| filen: 'cities/russia.csv'
100%|██████████| 2000/2000 [14:34<00:00,  2.29it/s]
ic| GREEDY_COST: np.float64(42334.16465744784)
    SIM_AN_COST: np.float64(36102.6069174666)
    EA_COST: np.float64(35457.67562739072)


In [68]:
filen= 'cities/us.csv'
instance = filen.split("/")[1].replace(".csv","")
ic(filen)
cities, dist_matrix = setup_data(filen)
CITIES = cities
DIST_MATRIX = dist_matrix
GREEDY_TSP, GREEDY_COST = greedy_tsp(cities, dist_matrix)
np.save(instance + "_greedy_tsp", GREEDY_TSP)
SIM_AN_SOL, SIM_AN_COST = simulated_annealing( GREEDY_TSP, 100, 0.999, 100000)
np.save(instance + "_localsearch_tsp", SIM_AN_SOL)
GENERATIONS = 3000
EA_TSP, EA_COST = ea_tsp(cities, GENERATIONS)
np.save(instance + "_ea_tsp", EA_TSP)
ic(GREEDY_COST,SIM_AN_COST, EA_COST)
None

ic| 

filen: 'cities/us.csv'
100%|██████████| 3000/3000 [1:24:53<00:00,  1.70s/it]
ic| GREEDY_COST: np.float64(48050.025864461386)
    SIM_AN_COST: np.float64(41462.17390227013)
    EA_COST: np.float64(42866.356201127455)


In [9]:
filen= 'cities/china.csv'
instance = filen.split("/")[1].replace(".csv","")
ic(filen)
cities, dist_matrix = setup_data(filen)
CITIES = cities
DIST_MATRIX = dist_matrix
GREEDY_TSP, GREEDY_COST = greedy_tsp(cities, dist_matrix)
np.save(instance + "_greedy_tsp", GREEDY_TSP)
SIM_AN_SOL, SIM_AN_COST = simulated_annealing( GREEDY_TSP, 100, 0.999, 100000)
np.save(instance + "_localsearch_tsp", SIM_AN_SOL)

GENERATIONS = 1000
EA_TSP, EA_COST = ea_tsp(cities, 1000)
np.save(instance + "_ea_tsp", EA_TSP)
ic(GREEDY_COST,SIM_AN_COST, EA_COST)
None

ic| filen: 'cities/china.csv'
  5%|▌         | 52/1000 [17:44<5:23:23, 20.47s/it]


KeyboardInterrupt: 